In [8]:
import cv2
import os
import numpy as np

# === Hardcoded settings ===
INPUT_PATH  = "37.png"              # change this as needed
OUTPUT_PATH = "binarized.png"       # where to save the binarized image
TXT_PATH    = "binarized_bits.txt"  # text file for binary matrix
METHOD      = "otsu"                # choose: "global", "adaptive_mean", "adaptive_gaussian", "otsu"
THRESH_VAL  = 127                   # used for global thresholding
MAX_VAL     = 255
ADAPTIVE_BLOCKSIZE = 11             # must be odd and >1
ADAPTIVE_C         = 2
# ===========================


def resize_image(img, size=(1024, 1024)):
    return cv2.resize(img, size, interpolation=cv2.INTER_AREA)


def binarize_image(input_path, output_path=None, txt_path=None):
    # Load image in grayscale
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Cannot read image: {input_path}")
    
    # Resize
    img = resize_image(img, (1024, 1024))
    
    # Choose thresholding method
    if METHOD == "global":
        ret, binary = cv2.threshold(img, THRESH_VAL, MAX_VAL, cv2.THRESH_BINARY)
        print(f"Global thresholding used threshold = {ret}")
    elif METHOD == "adaptive_mean":
        binary = cv2.adaptiveThreshold(
            img, MAX_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            ADAPTIVE_BLOCKSIZE,
            ADAPTIVE_C
        )
        print("Adaptive Mean thresholding used")
    elif METHOD == "adaptive_gaussian":
        binary = cv2.adaptiveThreshold(
            img, MAX_VAL,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            ADAPTIVE_BLOCKSIZE,
            ADAPTIVE_C
        )
        print("Adaptive Gaussian thresholding used")
    elif METHOD == "otsu":
        blur = cv2.GaussianBlur(img, (5,5), 0)
        ret, binary = cv2.threshold(blur, 0, MAX_VAL, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        print(f"Otsu’s thresholding used threshold = {ret}")
    else:
        raise ValueError(f"Unknown METHOD: {METHOD}")
    
    # Save binary image
    if output_path is None:
        base, ext = os.path.splitext(input_path)
        output_path = base + "_binarized.png"
    cv2.imwrite(output_path, binary)
    print(f"✅ Saved binarized image to: {output_path}")

    # Convert image pixels to 1s and 0s
    bits = (binary > 0).astype(np.uint8)  # 255 → 1, 0 → 0

    # Save as text file
    if txt_path is None:
        base, ext = os.path.splitext(output_path)
        txt_path = base + "_bits.txt"
    
    with open(txt_path, "w") as f:
        for row in bits:
            line = " ".join(map(str, row))
            f.write(line + "\n")
    
    print(f"✅ Saved binary matrix to: {txt_path}")

    return output_path, txt_path


if __name__ == "__main__":
    binarize_image(INPUT_PATH, OUTPUT_PATH, TXT_PATH)


Otsu’s thresholding used threshold = 127.0
✅ Saved binarized image to: binarized.png
✅ Saved binary matrix to: binarized_bits.txt


In [9]:
import cv2

# Load the binarized image (white objects on black background)
binary = cv2.imread("binarized.png", cv2.IMREAD_GRAYSCALE)

# Ensure it's truly binary (0/255)
_, binary = cv2.threshold(binary, 127, 255, cv2.THRESH_BINARY)

# Count connected components
num_labels, labels = cv2.connectedComponents(binary)

# Subtract 1 to ignore the background
object_count = num_labels - 1
print(f"Detected objects: {object_count}")


Detected objects: 64
